In [1]:
import findspark
findspark.init('/home/mj1e16/miniconda2/lib/python2.7/site-packages/pyspark')

In [2]:
import astroprov
import numpy as np
import os

from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table
from astropy.table import Column

import collections
import subprocess

import time
from multiprocessing import Pool
from multiprocessing.pool import ThreadPool

from functools import partial

from astropy.io import ascii
#from sqlalchemy import create_engine
import sqlite3
import mysql.connector
from pandas import DataFrame

import mysql.connector
from mysql.connector import Error
from mysql.connector import pooling

from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark import SparkConf
import datetime
from sqlalchemy import create_engine


In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/mj1e16/miniconda2/lib/python2.7/site-packages/pyspark/mysql-connector-java_8.0.16-1ubuntu16.04_all/usr/share/java/mysql-connector-java-8.0.16.jar  pyspark-shell'
config = SparkConf().setAll([('spark.executor.cores', '6'),('spark.cores.max', '6'),('spark.driver.memory','1g'),('spark.executor.memory', '500m')])


In [4]:
sc = SparkContext(appName='App',conf=config)
sqlContext = SQLContext(sc)
dataframe_mysql = sqlContext.read.format("jdbc").option("url", "jdbc:mysql://localhost/Kepler").option("driver", "com.mysql.jdbc.Driver").option("dbtable", "starlist").option("user", "mj1e16").option("password", "[sqlT1G3R]").load()
dataframe_mysql.registerTempTable('starlist')

In [5]:
def innerJoin(tableName,provDir,diffSize=1):
    temptableName = tableName[0:5]+tableName[-1]
    df = sqlContext.sql("SELECT NUMBER, EXT_NUMBER, FLUX_ISO, FLUXERR_ISO, BACKGROUND, THRESHOLD, FLUX_MAX, XPEAK_IMAGE, YPEAK_IMAGE,  X_IMAGE, Y_IMAGE, FWHM_IMAGE, ELLIPTICITY FROM {} INNER JOIN starlist ON {}.X_IMAGE BETWEEN starlist.X_POS_MIN AND starlist.X_POS_MAX AND {}.Y_IMAGE BETWEEN starlist.Y_POS_MIN AND starlist.Y_POS_MAX".format(temptableName,temptableName,temptableName))
    df.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='result_{}'.format(tableName),user='mj1e16',password='[sqlT1G3R]').mode('append').save()
    sqlContext.dropTempTable(temptableName)
    astroprov.provcall([tableName,'starlist'],['result_{}'.format(tableName)],"innerJoin_Python2Python_SQ_tmpl.provn","innerJoin",provDir)


def makeConfig(valList,provDir,tableName='table',defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',attributeList=['DETECT_THRESH','DETECT_MINAREA','BACK_FILTERSIZE','BACK_SIZE']):
    
    #workAroundList = [16,32,64,128,256,512]
    workAroundList = [1,3,5,7,9,11]
    ident = workAroundList.index(valList[-1])
    tableName = tableName[ident] # could just return a letter?
    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
    
    catLocFinder = 'CATALOG_NAME'
    catname = 'test'+str(ident)+'.cat'
    nameLoc = data.find(catLocFinder) + len(catLocFinder)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' '+catname+' ' + data[endLoc:]
    data = newData 
        
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData    
    
    
    confName = 'default_'+str(ident)+'.sex'
    with open(defaultDir+confName,'w') as f:
        f.write(data)
    astroprov.provcall(valList,[confName],"makeConfig_Python2Python_SQ_tmpl.provn","makeConfig",provDir)
    return(confName,tableName,catname)



def findObjects(confName,tableName,catName,provDir,defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',imagename='/home/mj1e16/iraf/editedImage5000.fits'):
    
    astroprov.provcall([confName,'/home/mj1e16/iraf/editedImage5000.fits'],[tableName],"findObjects_Python2DaoStarfidner_SQ_tmpl.provn","findObjects",provDir)

    os.chdir(defaultDir)
    subprocess.call(['sex',imagename,'-c',confName])
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df_spark = sqlContext.createDataFrame(df)
    temptableName = tableName[0:5]+tableName[-1]
    #print(temptableName)
    df_spark.registerTempTable(temptableName)

    #return df

def alltogethernow(valList,tableName,IMAGE,provDir):
    Names = makeConfig(valList,provDir,tableName=tableName)
    findObjects(Names[0],Names[1],Names[2],provDir,imagename=IMAGE) # confName tabName catname
    innerJoin(Names[1],provDir)


In [6]:
valList = [np.linspace(1,10,10),np.linspace(1,9,9),np.linspace(8,128,16),[1,3,5,7,9,11]] # astro start

In [7]:
def evaluateImage(valList,simImage,provDir,smallName='Poshak',atList=['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE']):

    p = ThreadPool(6)
    t0 = time.time()
    for x0 in range(len(valList[0])):
        for x1 in range(len(valList[1])):
                for x2 in range(len(valList[2])):
                    name = smallName+'_'+str(x0)+'_'+str(x1)+'_'+str(x2)+'_'
                    fullname = [name]*len(valList[3])
                    fullValList = []
                    for x3 in range(len(valList[3])):
                        fullname[x3] += str(x3)
                        print(fullname[x3])
                        fullValList.append([valList[0][x0],valList[1][x1],valList[2][x2],valList[3][x3]])

                        #alltogethernow(fullValList[x4],fullname,imageName,provDir)
                    try:
                        p.map(partial(alltogethernow,tableName=fullname,IMAGE=simImage,provDir=provDir),fullValList)
                    except RuntimeError:
                        with open('redos','a') as f:
                            bigString = simImage+','+str(fullname)+','+str(fullValList)
                            f.write(bigString)
                        print('Redo with valList = ',fullValList)
                        pass

    #def alltogethernow(valList,tableName,IMAGE,provDir):

    print(time.time()-t0)

In [ ]:
### Make function to replace starlist
evaluateImage(valList,'/home/mj1e16/Simages/testsimage_4_4.fits','/home/mj1e16/keplerPhotometry/provDump/44/selectiveMedian/',smallName='Poshak_44_1')

Poshak_44_1_0_0_0_0
Poshak_44_1_0_0_0_1
Poshak_44_1_0_0_0_2
Poshak_44_1_0_0_0_3
Poshak_44_1_0_0_0_4
Poshak_44_1_0_0_0_5
Poshak_44_1_0_0_1_0
Poshak_44_1_0_0_1_1
Poshak_44_1_0_0_1_2
Poshak_44_1_0_0_1_3
Poshak_44_1_0_0_1_4
Poshak_44_1_0_0_1_5


In [22]:
# make sim images and order properly
valList = [np.linspace(1,10,10),np.linspace(1,9,9),np.linspace(8,128,16),[1,3,5,7,9,11]] # astro start

In [15]:
#def evaluateImage(valList,simImage,provDir,smallName='Poshak',atList=['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE']):
smallName='Poshak'
atList=['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE']
simImage = '/home/mj1e16/Simages/testsimage_4_4.fits'
provDir = '~/dump/'
p = ThreadPool(6)
t0 = time.time()
tables = []
for x0 in range(1,2): #len(valList[0])):
    for x1 in range(1): #len(valList[1])):
            for x2 in range(1): #len(valList[2])):
                name = 'Test_'+smallName+'_'+str(x0)+'_'+str(x1)+'_'+str(x2)+'_'
                fullname = [name]*len(valList[3])
                fullValList = []
                for x3 in range(len(valList[3])):
                    fullname[x3] += str(x3)
                    print(fullname[x3])
                    fullValList.append([valList[0][x0],valList[1][x1],valList[2][x2],valList[3][x3]])

                    #alltogethernow(fullValList[x4],fullname,imageName,provDir)
                try:
                    tables.append(p.map(partial(alltogethernow,tableName=fullname,IMAGE=simImage,provDir=provDir),fullValList))
                except RuntimeError:
                    with open('redos','a') as f:
                        bigString = simImage+','+str(fullname)+','+str(fullValList)
                        f.write(bigString)
                    print('Redo with valList = ',fullValList)
                    pass

#def alltogethernow(valList,tableName,IMAGE,provDir):

print(time.time()-t0)

Test_Poshak_1_0_0_0
Test_Poshak_1_0_0_1
Test_Poshak_1_0_0_2
Test_Poshak_1_0_0_3
Test_Poshak_1_0_0_4
Test_Poshak_1_0_0_5
Empty DataFrame
Columns: [NUMBER, EXT_NUMBER, FLUX_ISO, FLUXERR_ISO, BACKGROUND, THRESHOLD, FLUX_MAX, XPEAK_IMAGE, YPEAK_IMAGE, X_IMAGE, Y_IMAGE, FWHM_IMAGE, ELLIPTICITY]
Index: []
Empty DataFrame
Columns: [NUMBER, EXT_NUMBER, FLUX_ISO, FLUXERR_ISO, BACKGROUND, THRESHOLD, FLUX_MAX, XPEAK_IMAGE, YPEAK_IMAGE, X_IMAGE, Y_IMAGE, FWHM_IMAGE, ELLIPTICITY]
Index: []
Empty DataFrame
Columns: [NUMBER, EXT_NUMBER, FLUX_ISO, FLUXERR_ISO, BACKGROUND, THRESHOLD, FLUX_MAX, XPEAK_IMAGE, YPEAK_IMAGE, X_IMAGE, Y_IMAGE, FWHM_IMAGE, ELLIPTICITY]
Index: []
Empty DataFrame
Columns: [NUMBER, EXT_NUMBER, FLUX_ISO, FLUXERR_ISO, BACKGROUND, THRESHOLD, FLUX_MAX, XPEAK_IMAGE, YPEAK_IMAGE, X_IMAGE, Y_IMAGE, FWHM_IMAGE, ELLIPTICITY]
Index: []
Empty DataFrame
Columns: [NUMBER, EXT_NUMBER, FLUX_ISO, FLUXERR_ISO, BACKGROUND, THRESHOLD, FLUX_MAX, XPEAK_IMAGE, YPEAK_IMAGE, X_IMAGE, Y_IMAGE, FWHM_IMAGE

In [16]:
(tables[0][0][1])

,index,X_POS,Y_POS,MAG,X_POS_MAX,Y_POS_MAX,X_POS_MIN,Y_POS_MIN
0,0,81.195,1.094,-0.060,None,None,None,None
1,1,245.725,1.936,-0.143,None,None,None,None
2,2,59.686,2.973,-2.222,None,None,None,None
3,3,151.423,3.443,-2.103,None,None,None,None
4,4,129.290,5.732,-0.915,None,None,None,None
5,5,260.531,6.172,-0.137,None,None,None,None
6,6,61.597,6.811,-0.620,None,None,None,None
7,7,252.638,8.114,-0.166,None,None,None,None
8,8,226.209,8.811,-0.986,None,None,None,None
9,9,138.623,9.330,-1.369,None,None,None,None


In [17]:
print(tables[0][0][1])

      index     X_POS     Y_POS    MAG X_POS_MAX Y_POS_MAX X_POS_MIN Y_POS_MIN
0         0    81.195     1.094 -0.060      None      None      None      None
1         1   245.725     1.936 -0.143      None      None      None      None
2         2    59.686     2.973 -2.222      None      None      None      None
3         3   151.423     3.443 -2.103      None      None      None      None
4         4   129.290     5.732 -0.915      None      None      None      None
5         5   260.531     6.172 -0.137      None      None      None      None
6         6    61.597     6.811 -0.620      None      None      None      None
7         7   252.638     8.114 -0.166      None      None      None      None
8         8   226.209     8.811 -0.986      None      None      None      None
9         9   138.623     9.330 -1.369      None      None      None      None
10       10   233.995    10.014 -0.358      None      None      None      None
11       11    78.133    10.056 -0.732      None    

In [12]:
df1 = sqlContext.sql("SELECT * FROM starlist")
df1 = df1.toPandas()

In [13]:
df1

,index,X_POS,Y_POS,MAG,X_POS_MAX,Y_POS_MAX,X_POS_MIN,Y_POS_MIN
0,0,81.195,1.094,-0.060,None,None,None,None
1,1,245.725,1.936,-0.143,None,None,None,None
2,2,59.686,2.973,-2.222,None,None,None,None
3,3,151.423,3.443,-2.103,None,None,None,None
4,4,129.290,5.732,-0.915,None,None,None,None
5,5,260.531,6.172,-0.137,None,None,None,None
6,6,61.597,6.811,-0.620,None,None,None,None
7,7,252.638,8.114,-0.166,None,None,None,None
8,8,226.209,8.811,-0.986,None,None,None,None
9,9,138.623,9.330,-1.369,None,None,None,None


In [47]:
tables[0][2]

(      NUMBER  EXT_NUMBER     FLUX_ISO  FLUXERR_ISO  BACKGROUND  THRESHOLD  \
 0          1           1     24.71090     4.329993   -0.795546    6.49499   
 1          2           1     40.66531     4.329993   -0.819244    6.49499   
 2          3           1     65.68936     7.499769   -1.173237    6.49499   
 3          4           1    912.63080    20.765920   -0.581027    6.49499   
 4          5           1    104.74230    12.247070   -0.688091    6.49499   
 5          6           1    681.28020    22.912170   -0.115483    6.49499   
 6          7           1    586.78640    22.912170   -0.757586    6.49499   
 7          8           1    760.10770    24.108380   -0.529828    6.49499   
 8          9           1    703.93100    24.494140   -0.555251    6.49499   
 9         10           1    924.07620    24.873920   -0.779079    6.49499   
 10        11           1    974.44190    26.338320   -0.676505    6.49499   
 11        12           1   1139.25700    25.247980   -0.617149 

In [ ]:
atList = ['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE']

#valList = [np.linspace(1.5,6,8),np.linspace(5,14,10),['AUTO'],np.linspace(1,10,10),[16,32,64,128,256,512]]
#astro = [(1,10,1),(1,9,1),[auto],(values from 7.2),(8,128,8)]
valList = [np.linspace(1,10,10),np.linspace(1,9,9),np.linspace(8,128,16),[1,3,5,7,9,11]] # astro start


p = ThreadPool(6)
t0 = time.time()

date = str(datetime.datetime.now())
day = date[0:10]
provstart = '/home/mj1e16/keplerPhotometry/provDump/'+day


for x in range(1,4):
    try:
        subprocess.call(['mkdir',provstart])
    except:
        print('day dir exists')
    try:
        subprocess.call(['mkdir',provstart+'/image'+str(x)])
        #os.mkdir(provstart+'/image'+str(x))
        #print(provstart+'/image'+str(x),'Sucess')
    except:
        print(provstart+'/image'+str(x),'Directory Already Exists')
        pass

#os.mkdir(provstart+'/image2')
#os.mkdir(provstart+'/image3')

imageTem = ['/home/mj1e16/iraf/kplr2009114174833_ffi-cal_TEMPLATE.fits']#,'/home/mj1e16/iraf/kplr2009114174833_ffi-cal_TEMPLATE_PLUS5000.fits']
imageIn = ['/home/mj1e16/iraf/kplr2010019225502_ffi-cal_DIFF_PLUS5000.fits'] #,'/home/mj1e16/iraf/kplr2010019225502_ffi-cal_DIFF.fits']
diffImage = '/home/mj1e16/iraf/simImage1.fits'
simulationImages = [[]]


for diff in range(len(imageIn)):
    for templ in range(len(imageTem)):
        for add in range(2):
            if add == 0:
                provDir = provstart+'/image{}_{}_{}/'.format(diff,templ,add)
                subprocess.call(['mkdir',provDir])
                addStars('starlist',imageIn[diff],provDir)
                differenceImage(imageTem[templ],imageIn[0],'/home/mj1e16/iraf/simImage1.fits',provDir)
                smallName = 'spark1'
        if imageNumber == 1:
            provDir = provstart+'/image2/'
            addStars('starlist',imageTem[0],provDir)
            imageName = differenceImage(imageDiff[0],imageTem[0],'/home/mj1e16/iraf/simImage2.fits',provDir)
            smallName = 'spark2'
        if imageNumber == 2:
            provDir = provstart+'/image3/'
            imageName = '/home/mj1e16/iraf/simpleDiffImagePLUS5000.fits'
            astroprov.provcall(['/home/mj1e16/iraf/kplr2010019225502_ffi-cal_DIFF.fits','/home/mj1e16/iraf/kplr2009114174833_ffi-cal_TEMPLATE.fits'],[imageName],"differenceImage_Python2Hotpants_SQ_tmpl.provn","differenceImage",provDir)
            addStars('starlist',imageName,provDir)
            smallName = 'spark3'
        for x0 in range(len(valList[0])):
            for x1 in range(len(valList[1])):
                    for x2 in range(len(valList[2])):
                        name = smallName+'_'+str(x0)+'_'+str(x1)+'_'+str(x2)+'_'
                        fullname = [name]*len(valList[3])
                        fullValList = []
                        for x3 in range(len(valList[3])):
                            fullname[x3] += str(x3)
                            print(fullname[x3])
                            fullValList.append([valList[0][x0],valList[1][x1],valList[2][x2],valList[3][x3]])

                            #alltogethernow(fullValList[x4],fullname,imageName,provDir)
                        try:
                            p.map(partial(alltogethernow,tableName=fullname,IMAGE=imageName,provDir=provDir),fullValList)
                        except RuntimeError:
                            print('Redo with valList = ',fullValList)
                            pass

#def alltogethernow(valList,tableName,IMAGE,provDir):

print(time.time()-t0)
